In [2]:
import xarray as xr
import pickle 
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#from eofs.xarray import Eof
from eofs.multivariate.iris import MultivariateEof
import iris
#import iris.plot as iplt
#import iris.quickplot as qplt
import datetime

anomalies_norm = xr.open_dataset("/home/srvx11/lehre/users/a1204101/AnalogMethod/data/ERA5/anom_all_1979-2018.nc",chunks={'time': 5000, 'lat':141, 'lon': 141})


In [3]:
result = pd.DataFrame(columns=['search_day', 'found_day', 'index'])
for doy in range(1,3):   # Loop über alle 366 dayofyears wenn: range(1,367):
    print('berechne doy: ', doy) # = Target Day
    
    # Bestimmen der benötigten doy im Zeitfenster
    days = np.arange(doy-10,doy+11)
    days[days<=0] = days[days<=0]+366
    days[days>366] = days[days>366]-366
    
    # Ausschneiden der 21 Tage (in allen Jahren), Größe: 21 Tage x 40 Jahre x 141x141
    anomalies_i_q = anomalies_norm.q.where(anomalies_norm.q.dayofyear.isin(days),drop=True).to_iris()
    anomalies_i_r = anomalies_norm.r.where(anomalies_norm.r.dayofyear.isin(days),drop=True).to_iris()
    anomalies_i_msl = anomalies_norm.msl.where(anomalies_norm.msl.dayofyear.isin(days),drop=True).to_iris()
    
    # solver-Objekt erzeugen
    solver = MultivariateEof([anomalies_i_q,anomalies_i_r,anomalies_i_msl])
    
    # immer mehr PCs bis 90% erklärte Varianz:
    N = 1
    var_f = 0
    while var_f < .9:
        var_f = np.sum(solver.varianceFraction(neigs=N).data)
        N = N+1
    pcs = solver.pcs(npcs=N)
    pcs_x = xr.DataArray.from_iris(pcs) # Umwandeln der PCs in xarray
    del pcs    
        
    for year in np.arange(1979,2019): # Loop über die 40 Jahre
        
        # Ausschneiden der einen benötigten Analyse (= ein Tag), Größe: 141x141
        single_q = anomalies_norm.q.sel(time=((anomalies_norm.time.dt.year == year)&(anomalies_norm.dayofyear == doy))).to_iris()
        single_r = anomalies_norm.r.sel(time=((anomalies_norm.time.dt.year == year)&(anomalies_norm.dayofyear == doy))).to_iris()
        single_msl = anomalies_norm.msl.sel(time=((anomalies_norm.time.dt.year == year)&(anomalies_norm.dayofyear == doy))).to_iris()

        # Berechnen der Pseudo-PCs für den einen Tag
        pseudo_pcs = solver.projectField([single_q, single_r, single_msl], neofs=N)
        
        # Minimieren der Norm (=Suchen des analogen Tags = 'found_day')
        found_idx=np.argmin(np.sum(np.sqrt((pcs_x.sel(time=~(pcs_x.time.dt.year == year)).data - pseudo_pcs.data)**2),axis=1)) # Index im pcs
        found_total_idx=anomalies_i_q.coord('time').points[found_idx] # Index im gesamter Anomalies
        
        search_day = anomalies_norm.sel(time=((anomalies_norm.time.dt.year == year)&(anomalies_norm.dayofyear == doy))).time[0].data # 'aktueller Tag'
        found_day = pcs_x.sel(time=~(pcs_x.time.dt.year == year)).isel(time=found_idx).time.data
        
        result = result.append(pd.DataFrame([[pd.to_datetime(search_day),pd.to_datetime(found_day),found_total_idx]],columns=['search_day', 'found_day', 'index']))

print(result.sort_values('search_day'))
pickle.dump(result.sort_values('search_day'), open( "save_found_dates.p", "wb" ) )
# öffnen mit XXX = pickle.load( open( "save_found_dates.p", "rb" ) )

berechne doy:  1
berechne doy:  2
   search_day  found_day  index
0  1979-01-01 2003-01-04   8404
0  1979-01-02 1995-01-12   5490
0  1980-01-01 1981-01-05    369
0  1980-01-02 1985-01-02   1827
0  1981-01-01 2005-01-02   9133
0  1981-01-02 1986-12-28   2553
0  1982-01-01 2010-01-01  10958
0  1982-01-02 2011-01-12  11334
0  1983-01-01 2005-01-01   9132
0  1983-01-02 2007-01-06   9867
0  1984-01-01 1980-12-31    730
0  1984-01-02 1988-12-23   2933
0  1985-01-01 1980-01-02    366
0  1985-01-02 1980-01-02    366
0  1986-01-01 1985-12-31   2556
0  1986-01-02 1994-01-04   5117
0  1987-01-01 1986-12-31   2921
0  1987-01-02 2002-12-31   8400
0  1988-01-01 1998-01-03   6576
0  1988-01-02 1994-12-29   5475
0  1989-01-01 1988-12-31   3652
0  1989-01-02 2016-12-29  13512
0  1990-01-01 1989-12-31   4017
0  1990-01-02 1992-12-29   4746
0  1991-01-01 1999-01-03   6942
0  1991-01-02 2004-01-12   8777
0  1992-01-01 2005-01-06   9136
0  1992-01-02 2017-12-24  13871
0  1993-01-01 1991-12-31   4747
0  199